In [ ]:
%matplotlib inline
import sys
sys.path.append('..')
from sub_projects.ray_optimization.configuration import params_to_func
from ray_optim.plot import Plot
import torch
from matplotlib import pyplot as plt

In [ ]:
fig = Plot.plot_data([torch.randn(3, 1000, 2) for i in range(3)])
plt.figure(fig)

In [ ]:
samples_count = 1
compensated = [torch.randn(3, 1000, 2) * 0.1 + 0.1 for _ in range(samples_count)]
target = [torch.randn(3, 1000, 2) * 0.1 - 1 for _ in range(samples_count)]
without_compensation = compensated
fig = Plot.fixed_position_plot(
    compensated, target, without_compensation, (-2.0, 2.0), (-2.0, 2.0), epoch=12
)
plt.figure(fig)

In [ ]:

parameters = {
            "x_var": [-5., 5.],
            "y_var": [0., 10.],
            }
search_space = params_to_func(parameters)()
real_params = params_to_func({"x_var": 0, "y_var":5})()
predicted_params = params_to_func({"x_var": -2.5, "y_var":10.})()
Plot.plot_param_comparison(epoch=42, real_params=real_params, predicted_params=predicted_params, search_space=search_space)

In [ ]:
samples_count = 1
compensated = [torch.randn(3, 1000, 2) * 0.1 + 0.1 for _ in range(samples_count)]
target = [torch.randn(3, 1000, 2) * 0.1 +0.1*i for i in range(samples_count)]
without_compensation = compensated
fig = Plot.compensation_plot(
    compensated, target, without_compensation, epoch=42
)
plt.figure(fig)

In [ ]:
parameters = {
    "x_var": [-5.0, 5.0],
    "y_var": [0.0, 10.0],
}
search_space = params_to_func(parameters)()
real_params = params_to_func({"x_var": 0, "y_var": 5})()
predicted_params = params_to_func({"x_var": -2.5, "y_var": 10.0})()

Plot.plot_param_comparison(
    epoch=42,
    real_params=real_params,
    predicted_params=predicted_params,
    search_space=search_space,
)